In [48]:
import torch
import matplotlib

In [49]:
words=open('names.txt', 'r').read().splitlines()

In [50]:
len(words)

32033

In [51]:
b={}
for w in words:
    chs = ['<S>'] + list(w) + ['<E>']
    for c1, c2 in zip(chs, chs[1:]):
        bigram = (c1, c2)
        b[bigram] = b.get(bigram,0) + 1

In [52]:
N=torch.zeros((28,28), dtype=torch.int32)

In [53]:
chars = sorted(list(set(''.join(words))))
stoi={s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()} 

In [54]:
for w in words:
    chs = ['.'] + list(w) + ['.']
    for c1, c2 in zip(chs, chs[1:]):
        ix1 = stoi[c1]
        ix2 = stoi[c2]
        N[ix1, ix2] += 1

In [63]:
# Creating a training set
xs,ys = [], []
for w in words[:1]:
    chs = ['.'] + list(w) + ['.']
    for c1, c2 in zip(chs, chs[1:]):

        idx1 = stoi[c1]
        idx2 = stoi[c2]

        xs.append(idx1)
        ys.append(idx2)
xs=torch.tensor(xs)
ys=torch.tensor(ys)

In [64]:
import torch.nn.functional as F

xenc = F.one_hot(xs, num_classes=27).float()

In [66]:
xenc

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [76]:
W = torch.randn((27, 27), requires_grad=True)

In [77]:
# forward pass
logits = xenc @ W
counts = logits.exp()
probs = counts/counts.sum(1, keepdims=True)
loss=-probs[torch.arange(5) , ys].log().mean()

In [78]:
print(loss.item())

3.8995609283447266


In [ ]:
# backward pass
W.grad = None
loss.backward()  

In [ ]:
W.data += 0.1 * W.grad